# Stacks

## Definition 

- Collection of objects inserted and removed according to last-in-first-out (LIFO) principle
- A user may insert objects into a stack at any time but may only access the most recently inserted object that remains at the top of stack.

## Examples

- Web browsers storing the recently opened web addresses in a stack to implement the back button functionality
- Undo mechanism in text editors
- Validation of arithmetic expressions and HTML by testing of matching pairs of delimiters

## ADT

A stack ADT with an instance S will support the following methods

- S.push(e): Adds the element e on top of stack
- S.pop(): Removes and returns the top element. Error occurs if stack is empty

Accessor methods

- S.top(): Returns a reference to top element without removing it. Error occurs if stack is empty.
- S.is_empty(): Returns True if stack does not contain any elements. Otherwise False.
- len(S): Returns number of elements in the stack



In [1]:
class Empty(Exception):
    """Error attempting to access an element from an empty container."""

    pass


class ArrayStack:
    def __init__(self):
        self._array = []

    def push(self, element):
        self._array.append(element)

    def pop(self):
        if self.is_empty():
            raise Empty
        return self._array.pop()

    def top(self):
        if self.is_empty():
            raise Empty
        return self._array[-1]

    def is_empty(self):
        return len(self._array) == 0

    def __len__(self):
        return len(self._array)

## Analysing ArrayStack Implementation

- The implementations for top(), is_empty() and len take constant time in the worst case.
- push() and pop() also take O(1) time which is amortized bound.
  - A typical call to either of these methods will take constant time but there is an occasionally an O(n) time worst case where an operation causes the list to resize its internal array.

### Avoiding Amortization by Reserving Capacity

- TODO

## More examples

- A stack can be used as a general tool to reverse a data sequence. For example : we might want to print lines of a file in a reverse.
- For testing pairs of matching delimiters
  - Validating an arithmetic expression
  - Validing HTML

In [2]:
def reverse_file(filename):
    stack = ArrayStack()
    with open(filename) as f:
        for line in f:
            line = line.rstrip("\n")
            stack.push(line)

    with open(filename, "w") as f:
        while not stack.is_empty():
            f.write(stack.pop() + "\n")

In [3]:
# TODO : Reversing elements within a stack

In [4]:
# O(n)
def validate_expr(expr):
    pairs = {"}": "{", "]": "[", ")": "("}
    closing = list(pairs.keys())
    opening = list(pairs.values())

    stack = ArrayStack()

    # O(n)
    for char in expr:
        if char in opening:
            # O(1)
            stack.push(char)

        elif char in closing:
            # O(1)
            if stack.is_empty():
                return False
            else:
                # O(1)
                if stack.pop() != pairs[char]:
                    return False

    return True


validate_expr("(5+5)(((((()))))){}{}{}{}{}{}{}{}{{{{{{{{22+22}}}}}}}}[]")

True

In [13]:
def validate_html(html):
    tag_start = html.find("<")
    stack = ArrayStack()

    while tag_start != -1:
        tag_end = html.find(">", tag_start)

        tag = html[tag_start : tag_end + 1]

        if "/" not in tag:
            stack.push(tag)

        else:
            if stack.is_empty():
                return False
            else:
                if tag != "</" + stack.pop()[2:]:
                    return False

        tag_start = html.find("<", tag_end)

    return True


with open("files/html.txt") as f:
    html = f.read()


validate_html(html)

False

# Queues

- Collection of objects inserted and removed according to first-in-first-out (FIFO) principle.
- Elements can be inserted at any time but only the element that has been in the queue the longest time can be next removed.

## The queue ADT

- q.enqueue(e): adds element to the back of the queue.
- q.dequeue(): returns and removes the first element of the queue. Error occurs if queue is empty.

Accessor Methods 

- q.first(): Returns the first element without removing it
- q.is_empty(): Returns True if queue is empty 
- len(q): Returns length of queue

## Array based queue implementation

- We could enqueue e by calling append(e) to add it to the end of list. 
- We could use pop(0) to remove the first element as opposed to pop() when dequeuing.
- The above approach in inefficient.
  - When pop is called with non-default index, a loop is executed to shift all elements to the left so as to fill the hole in the sequence caused by pop.
  - Therefore, pop(0) always causes worst case behaviour of O(n)
- We could avoid pop(0) by simply making the dequeued element as None and maintaining the index of the front element separately. 
- Over time, the size of this list will grow to O(m) where m is the number of enqueue operations since the creation of the queue rather than the current number of elements in the queue.
- This again can be solved by using the array circularly. 
- We will allow the front of the queue to drift rightward and we allow the contents of the queue to wrap around the underlying queue.

In [92]:
class ArrayQueue:
    DEFAULT_CAPACITY = 10

    def __init__(self):
        self._array = [None] * self.DEFAULT_CAPACITY
        self._front = 0
        self._size = 0

    def _resize(self, capacity):
        old = self._array

        self._array = [None] * capacity

        for i in range(self._size):
            self._array[i] = old[(self._front + i) % len(old)]

        self._front = 0

    def enqueue(self, element):
        if self._size == len(self._array):
            self._resize(2 * self._size)

        self._array[(self._front + self._size) % len(self._array)] = element
        self._size += 1
        print(self)

    def dequeue(self):
        if self.is_empty():
            raise Empty

        element = self._array[self._front]
        self._array[self._front] = None
        self._front = (self._front + 1) % len(self._array)
        self._size -= 1

        if self._size == (len(self._array) / 2):
            self._resize(self._size)

        print(self)
        return element

    def is_empty(self):
        return self._size == 0

    def first(self):
        if self.is_empty():
            raise Empty

        element = self._array[self._front]
        print(f"first element = {element}")
        return element

    def __len__(self):
        return self._size

    def __repr__(self):
        return f"{self._array} of size = {self._size} and front index = {self._front}"


q = ArrayQueue()

## Analyzing the above queue implementation

- Exception of the `_resize` utility, all method rely on a constant number of statements. 
- Therefore, each method runs in O(1) time except for enqueue and dequeue which have amortized bounds of O(1) time. 
- The bounds for enqueue and dequeue are amortized due to the resizing of the array. 
- The space usage is O(n) where n is the current number of elements in the queue.


- TODO check actual python queue implementation

# Double-Ended Queues / Deck

- The deck is more general data structure than stack and queue - it supports insertion and removal at both ends of the queue.

## Deck ADT

- d.add_first(e)
- d.add_last(e)
- d.delete_first()
- d.delete_last()

**Accessor Methods**

- d.first()
- d.last()
- d.is_empty()
- len(d)

The deck ADT can be implemented using the similar circular array approach.

## Python collections deque

- Chosen to be consistent with the naming conventions in the list class
- deque also supports an optional maxlen parameter to force a fixed length deque
  - However, if append to either end exceeds the allowed length it does not throw an error it just causes one element to be dropped from the other side.
- The current deque is implemented with a hybrid approach that uses aspects of circular arrays but organized into blocks that are themselves organized in a doubly linked list. 
  - The deque class guarantees O(1) time operations at either end but O(n) worst case when using index notations near middle of deque.
- Following is an example of usability.


In [101]:
from collections import deque


d = deque()

for i in range(10):
    # Appends to end
    d.append(i)

for i in range(10, 20):
    # Appends to first
    d.appendleft(i)


# Access first element
print(d[0])

# Removes first element
d.popleft()

# Access last element
print(d[-1])

# Removes last element
d.pop()

# Access arbitrary element by index
print(d[5])

# Modify arbitrary element by index
d[5] = 5


# clear all contents
# d.clear()

print(d)

# circularly shift rightward k steps
d.rotate(5)

print(d)

# remove first matching element
d.remove(15)

print(d)

# count number of matches 
print(d.count(5))

print(d)

19
9
13
deque([18, 17, 16, 15, 14, 5, 12, 11, 10, 0, 1, 2, 3, 4, 5, 6, 7, 8])
deque([4, 5, 6, 7, 8, 18, 17, 16, 15, 14, 5, 12, 11, 10, 0, 1, 2, 3])
deque([4, 5, 6, 7, 8, 18, 17, 16, 14, 5, 12, 11, 10, 0, 1, 2, 3])
2
deque([4, 5, 6, 7, 8, 18, 17, 16, 14, 5, 12, 11, 10, 0, 1, 2, 3])


In [ ]:
# TODO Exercises